In [1]:
import os
from dotenv import load_dotenv

# 1. 修复导包：统一使用核心库，并直接使用原生的 pydantic
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 2. 必备起手式：强制加载本地 API 密钥
load_dotenv(override=True)

# 3. 实例化大模型，极其关键：温度必须设为 0！
model = OpenAI(temperature=0)

# =================================================================
# 核心魔法：定义 Pydantic 数据模型 (也就是你强制要求的数据结构)
# =================================================================
class Ticket(BaseModel):
    # Field 里面的 description 就是在教大模型：“这个字段应该填什么”
    date: str = Field(description="show date (例如: Friday)")
    time: str = Field(description="show time")
    theater: str = Field(description="theater name")
    count: int = Field(description="number of tickets (必须是整数数字)")
    movie: str = Field(description="preferred movie")

# 初始化 Pydantic 解析器 (注意老师代码里有个小拼写错误 parcer，我帮你订正为 parser 了)
parser = PydanticOutputParser(pydantic_object=Ticket)

# 编写提示词模板 (把解析器的格式要求强行注入)
ticket_template = """
        Book us a movie ticket for two this Friday at 6:00 PM.
        Choose any theater, it doesn't matter. Send the confirmation by email.
        Our preferred movie is: {query}
        
        Format instructions:
        {format_instructions}
"""

prompt = PromptTemplate(
    template=ticket_template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 填入你要看的电影名字，生成最终的指令
input_prompt = prompt.format_prompt(query="The Batman")
print("✉️ 发给大模型的最终指令：\n", input_prompt.to_string())
print("=" * 60)

# 呼叫大模型，获取纯文本回复
output = model.invoke(input_prompt.to_string())
print("📦 大模型返回的原始纯文本:\n", output.strip())
print("=" * 60)

# =================================================================
# 见证奇迹：Pydantic 解析！
# =================================================================
reservation = parser.parse(output)

print("✨ 解析后的终极结果 (Pydantic 对象):")
print(reservation)
print("\n🔍 检查数据类型:", type(reservation))

# 像调用普通 Python 对象的属性一样，极其优雅地提取数据！
print("-" * 30)
print(f"🎬 电影名称: {reservation.movie}")
print(f"🎫 订票数量: {reservation.count} (数据类型是: {type(reservation.count)})")
print(f"📅 观影时间: {reservation.date} {reservation.time}")

✉️ 发给大模型的最终指令：
 
        Book us a movie ticket for two this Friday at 6:00 PM.
        Choose any theater, it doesn't matter. Send the confirmation by email.
        Our preferred movie is: The Batman
        
        Format instructions:
        The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date": {"description": "show date (例如: Friday)", "title": "Date", "type": "string"}, "time": {"description": "show time", "title": "Time", "type": "string"}, "theater": {"description": "theater name", "title": "Theater", "type": "string"}, "count": {"description": "number of tickets 